# 方法
* 传统的面向对象语言的分派只基于第一个参数，Julia使用函数的所有参数来决定调用哪一个方法被称为多重分派。

* 定义一个有多个方法的函数，只需简单定义这个函数多次，使用不同的参数数量和类型。

* methods(函数名)可以查看函数的方法和特征。

* 没有::的类型声明，方法参数的类型默认为Any，这就意味着没有约束，因为Julia中的所有的值都是抽象类型Any的实例。

## 方法歧义
* 在一系列的函数方法定义时有可能没有单独的最专用的方法适用于参数的某些组合.
```Julia
g(x::Float64,y)=2x+y
g(x,y::Float64)=x+2y
g(2.0,3.0)#报错，方法歧义
#建议先定义没有歧义的方法
```
## 参数方法
方法定义可以视需要存在限定特征的类型参数.<br/>
参数方法允许与where表达式同样的语法用来写类型（参见 UnionAll 类型）。如果只有一个参数，封闭的大括号（在where {T}中）可以省略，但是为了清楚起见推荐写上。多个参数可以使用逗号隔开，例如where {T, S <: Real}，或者使用嵌套的where来写，例如where S<:Real where T。

## 重定义方法
当重定义一个方法或者增加一个方法时，知道这个变化不会立即生效很重要。这是Julia能够静态推断和编译代码使其运行很快而没有惯常的JIT技巧和额外开销的关键。实际上，任意新的方法定义不会对当前运行环境可见，包括Tasks和线程（和所有的之前定义的@generated函数）。
```Julia
function tryeval()
    @eval newfun()=1
    newfun()
end

tryeval()#报错
newfun()#1
```
调用Base.invokelatest函数即时更新函数方法。      
## 使用参数方法设计样式
虽然复杂的分派逻辑对于性能或者可用性并不是必须的，但是有时这是表达某些算法的最好的方法。
### 从超类型中提取出类型参数
返回AbstractArray的任意子类型的元素类型T
```Julia
abstract type AbstractArray{T,N} end
eltype(::Type{<:AbstractArray{T}}) where {T} =T
#三角分派，注意T是一个UnionAll类型
#如eltype(Array{T} where T <: Integer),会返回Any

#在Julia v0.6中的三角分派到来之前唯一正确的方法
abstract type AbstractArray{T,N} end
eltype(::Type{AbstractArray})=Any
eltype(::Type{AbstractArray{T}}) where {T}=T
eltype(::Type{AbstractArray{T,N}}) where {T,N}=T
eltype(::Type{A}) where {A<:AbstractArray}=eltype(supertype(A))

#对于参数T需要被更严格匹配的情况
eltype(::Type{AbstractArray{T,N} where {T<:S,N<:M}}) where {M,S}=Any
eltype(::Type{AbstractArray{T,N} where {T<:S}}) where {N,S}=Any
eltype(::Type{AbstractArray{T,N} where {N<:M}}) where {M,T}=T
eltype(::Type{AbstractArray{T,N}}) where {T,N}=T
eltype(::Type(A)) where {A<:AbstractArray}=eltype(supertype(A))
```
### 用不同的类型参数构建相似的类型
```Julia
input=convert(AbstractArray{Eltype},input)
output=similar(input,Eltype)
#Create an uninitialized mutable array with the given element type and size
#similar(array, [element_type=eltype(array)], [dims=size(array)])
copywitheltype(input,Eltype)=copyto!(similar(input,Eltype),input)

#Copy N elements from collection src starting at offset so, to array dest starting at offset do. Return dest.copyto!(dest, do, src, so, N)
```
在算法需要输入数组的拷贝的情况下，convert使无法胜任的，因为返回值可能只是原始输入的别名。把similar（构建输出数组）和copyto!（用输入数据填满）结合起来是需要给出输入参数的可变拷贝的一个范用方法
### 迭代分派
为了分派多层参数列表，最好是将每层分派分离到不同的函数中。例如，试图一次分派一个数组的元素类型常常会引发歧义，但是，如果首先分派容器类型，然后递归地分派基于与eltype的更具体的方法。例如：
```Julia
+(a::Matrix,b::Matrix)=map(+,a,b)
+(a,b)=(promote(a,b)...)
+(a::Float64,b::Float64)=Core.add(a,b)
```
### Trait-based dispatch
迭代分派的一个自然扩展就是增加一个方法选择层，这个方法选择层允许分派独立于类型层次的类型集合，我们可以通过写出类型的Union来构建这个集合，但如此一来由于Union类型不可更改，所以这个集合也不可扩展，此时，我们可以选择通过“Holy-Trait”进行扩展。

这个模式是通过定义一个生成函数实现的，这个生成函数针对每个函数参数归属的trait-set计算一个单元素值或类型。如果这个函数是pure，那这种模式与正常的分派无异。

map和promote函数就是采用了trait分派。当对一个矩阵遍历，如执行map函数，一个很重要的问题就是采用什么顺序，当AbstractArray子类型执行Base.IndexStyle trait,其他函数如map就可以根据这个信息选择最好的算法。这意味着无需针对每个子类型执行相应的map方法，因为生产（函数）定义+trait类就可以使系统选择最快的版本。例如
```Julia
map(f, a::AbstractArray, b::AbstractArray) = map(Base.IndexStyle(a, b), f, a, b)
# generic implementation:
map(::Base.IndexCartesian, f, a::AbstractArray, b::AbstractArray) = ...
# linear-indexing implementation (faster)
map(::Base.IndexLinear, f, a::AbstractArray, b::AbstractArray) = ...
```
promote机制中的标量+也使用了基于trait的方式，它使用promote_type，返回最优的一般类型去计算某运算（当给定两个运算对象类型），这可以将针对每对可能参数类型执行所有函数，简化为执行一个转换操作即将每个类型转换为一个常规类型，加一个逐对提升规则表格。
### 输出类型计算
trait-based提升能够为们计算矩阵操作的输出元素类型提供帮助，在执行原始操作时，如+，我们使用promote-typehas计算想要的输出类型。对于复杂的矩阵函数操作，针对复杂的操作结果计算期望的返回类型是十分有必要的，其步骤如下：
1. 编写一个小函数op来表示算法核心中使用的运算的集合
2. 使用promote_op(op, argument_types...)计算结果矩阵的元素类型R， 这里argument_types是通过应用到每个输入数组的eltype计算的。
3. 创建类似于similar(R, dims)的输出矩阵，这里dims是输出矩阵的预期维度数。

In [25]:
same_type(x::T,y::T) where {T}=true#相当于where{T<:Any}
same_type(x,y)=false
same_type(1,2)#true
same_type(1,2.0)#false
#这样的定义对应着那些类型名是 UnionAll 类型的方法

myappend(v::Vector{T},x::T) where {T}=[v...,x]
myappend([1,2,3],4)
#myappend([1,2,3],2.5)#报错

###
same_type_numeric(x::T,y::T) where{T<:Number}=true
same_type_numeric(x::Number,y::Number)=false
same_type_numeric(1,2)#ture
same_type_numeric(1,2.0)#false


####重定义
function tryeval()
    @eval newfun()=1
    newfun()
end
tryeval()#运行会报错，除非newfun()已被调用

#####Base.invokelatest可以即时更新函数
function tryeval2()
    @eval newfun2()=2
    Base.invokelatest(newfun2)
end
tryeval2()

#####重定义2
f(x)="original definition"
g(x)=f(x)
t=@async f(wait());yield();
f(x::Int)="definition for Int"

f(x::Type{Int})="definition for Type{Int}"
f("b")

f(1)

f(Int)
fetch(schedule(t,Int))#查看针对某个数据类型的方法,但只能执行一次



"definition for Type{Int}"